In [1]:
from functions.metabase import (
    make_metabase_session, 
    get_metabase_cards, 
    parse_metabase_cards, 
    transform_metabase_cards,
    get_cards_to_archive, 
    archive_batch
)
import os
from dotenv import load_dotenv

load_dotenv()

METABASE_URL = "https://sfr3.metabaseapp.com"
METABASE_API_KEY = os.environ.get('METABASE_API_KEY')


In [2]:
session = make_metabase_session(METABASE_API_KEY)

In [3]:
cards = get_metabase_cards(METABASE_URL, session)

In [4]:
df = parse_metabase_cards(cards)
df.head()

,card_id,collection_id,card_name,card_sql,db_id,created_at,created_by,last_updated_at,last_updated_by,last_used_at,archived
0,571,92.0,$0 line item explorer,with projects as ( \n SELECT distinct co.pr...,35,2023-04-25T21:40:31.977831Z,Andrew Faircloth,2023-06-09T02:28:15.248034Z,anthony.marinelli@sfr3.com,2024-08-29T21:04:33.448571Z,False
1,1420,128.0,0-rent leases,"select \nunitnumber as prop_id\n, l.createddat...",35,2023-05-30T16:15:42.214214Z,Andrew Nelson,2023-05-30T16:27:11.44031Z,andrew.nelson@sfr3.com,2024-08-29T21:04:33.448571Z,False
2,471,128.0,0 rent leases in buildium,select * \n\nfrom {{#431-basic-lease-info}}\n\...,35,2023-04-18T16:44:15.272299Z,Andrew Nelson,2023-04-18T18:54:32.4301Z,andrew.nelson@sfr3.com,2024-08-29T21:04:33.448571Z,False
3,10137,3070.0,$0 turn moveout reasons,"SELECT y.propid, y.project_id, y.metro, y.most...",35,2024-08-27T10:18:01.068658Z,Amartya Sengupta,2024-08-27T18:12:47.087135Z,amartya@sfr3.com,2024-09-23T16:05:06.915891Z,False
4,8362,NaN,100 Spokane Drive,None,35,2024-04-15T13:05:43.791332Z,Timothy Cha,2024-09-01T04:25:32.039323Z,timothy.cha@sfr3.com,2024-09-01T04:25:23.354215Z,False


In [5]:
df_transformed = transform_metabase_cards(df)
df_transformed.head()

,card_id,collection_id_x,card_name_x,card_sql_x,db_id_x,created_at_x,created_by_x,last_updated_at_x,last_updated_by_x,last_used_at_x,...,raw_stitch,raw_fivetran,raw_airbyte,raw_portable,raw_dataddo,flagged,days_since_last_usage,days_since_last_update,update_category,usage_category
0,571,92.0,$0 line item explorer,with projects as ( \n SELECT distinct co.pr...,35,2023-04-25T21:40:31.977831Z,Andrew Faircloth,2023-06-09T02:28:15.248034Z,anthony.marinelli@sfr3.com,2024-08-29T21:04:33.448571Z,...,NaN,NaN,NaN,NaN,NaN,False,113,560,since 500-599 days,since 110-119 days
1,1420,128.0,0-rent leases,"select \nunitnumber as prop_id\n, l.createddat...",35,2023-05-30T16:15:42.214214Z,Andrew Nelson,2023-05-30T16:27:11.44031Z,andrew.nelson@sfr3.com,2024-08-29T21:04:33.448571Z,...,False,False,False,True,False,True,113,570,since 500-599 days,since 110-119 days
2,471,128.0,0 rent leases in buildium,select * \n\nfrom {{#431-basic-lease-info}}\n\...,35,2023-04-18T16:44:15.272299Z,Andrew Nelson,2023-04-18T18:54:32.4301Z,andrew.nelson@sfr3.com,2024-08-29T21:04:33.448571Z,...,NaN,NaN,NaN,NaN,NaN,False,113,612,since 600-699 days,since 110-119 days
3,10137,3070.0,$0 turn moveout reasons,"SELECT y.propid, y.project_id, y.metro, y.most...",35,2024-08-27T10:18:01.068658Z,Amartya Sengupta,2024-08-27T18:12:47.087135Z,amartya@sfr3.com,2024-09-23T16:05:06.915891Z,...,NaN,NaN,NaN,NaN,NaN,False,88,115,since 100-199 days,since 80-89 days
4,8362,NaN,100 Spokane Drive,None,35,2024-04-15T13:05:43.791332Z,Timothy Cha,2024-09-01T04:25:32.039323Z,timothy.cha@sfr3.com,2024-09-01T04:25:23.354215Z,...,NaN,NaN,NaN,NaN,NaN,False,110,110,since 100-199 days,since 100-109 days


In [6]:
df_transformed.shape

(7554, 36)

In [7]:
df_to_archive = get_cards_to_archive(df_transformed)
df_to_archive.head()

,card_id,collection_id_x,card_name_x,card_sql_x,db_id_x,created_at_x,created_by_x,last_updated_at_x,last_updated_by_x,last_used_at_x,...,raw_stitch,raw_fivetran,raw_airbyte,raw_portable,raw_dataddo,flagged,days_since_last_usage,days_since_last_update,update_category,usage_category
1,1420,128.0,0-rent leases,"select \nunitnumber as prop_id\n, l.createddat...",35,2023-05-30T16:15:42.214214Z,Andrew Nelson,2023-05-30T16:27:11.44031Z,andrew.nelson@sfr3.com,2024-08-29T21:04:33.448571Z,...,False,False,False,True,False,True,113,570,since 500-599 days,since 110-119 days
156,5067,697.0,814 Dover Ave listing history,select parse_json(unit)['UNITNUMBER']::varchar...,35,2023-08-24T18:39:11.727188Z,Andrew Nelson,2023-09-08T20:54:43.506816Z,andrew.nelson@sfr3.com,2024-08-29T21:04:33.448571Z,...,False,False,False,False,False,True,113,469,since 400-499 days,since 110-119 days
179,4786,NaN,AAPM Moveouts,with lease as (\n select \n prop_id ...,35,2023-08-02T16:33:57.572539Z,Taylor Geib,2023-10-11T18:10:48.138526Z,taylor.geib@sfr3.com,2024-08-29T21:04:33.448571Z,...,False,False,False,True,False,True,113,436,since 400-499 days,since 110-119 days
181,4549,NaN,AAPM Reversals,with lease as (\n select \n id::stri...,35,2023-07-13T20:58:57.330229Z,Andrew Dallas,2023-07-26T06:48:07.734237Z,andrew@sfr3.com,2024-08-29T21:04:33.448571Z,...,False,False,False,True,False,True,113,513,since 500-599 days,since 110-119 days
184,4858,NaN,AAPM Transition Upcoming Turns - LUK / CMH / DAY,"with AAPM_transition as (\nSELECT \npropid \n,...",35,2023-08-09T15:22:25.279421Z,Taylor Geib,2024-03-19T13:24:27.412942Z,taylor.geib@sfr3.com,2024-08-29T21:04:33.448571Z,...,False,False,False,True,False,True,113,276,since 200-299 days,since 110-119 days


In [9]:
final_output = archive_batch(df_to_archive, session, METABASE_URL)
final_output

Archiving card_id 1420
Card 1420 succesfully archived
Archiving card_id 5067
Card 5067 succesfully archived
Archiving card_id 4786
Card 4786 succesfully archived
Archiving card_id 4549
Card 4549 succesfully archived
Archiving card_id 4858
Card 4858 succesfully archived
Archiving card_id 4613
Card 4613 succesfully archived
Archiving card_id 5300
Card 5300 succesfully archived
Archiving card_id 3368
Card 3368 succesfully archived
Archiving card_id 3731
Card 3731 succesfully archived
Archiving card_id 2615
Card 2615 succesfully archived
Archiving card_id 748
Card 748 succesfully archived
Archiving card_id 749
Card 749 succesfully archived
Archiving card_id 1432
Card 1432 succesfully archived
Archiving card_id 963
Card 963 succesfully archived
Archiving card_id 687
Card 687 succesfully archived
Archiving card_id 688
Card 688 succesfully archived
Archiving card_id 444
Card 444 succesfully archived
Archiving card_id 6795
Card 6795 succesfully archived
Archiving card_id 5291
Card 5291 succesf

'104 cards were archived. 0 cards were NOT archived. Cards to try again: []'